## 4. Prep Statistics CSV

Use this notebook to prepare hydrologic statistic attributes for addition to the segments shapefile. The stats are saved to a CSV. These stats are a subset of the full dataset designed for quick display on the webapp via WMS / WFS. Unless otherwise noted, we are providing the difference between static landcover, Maurer model, historical scenario, 1976-2005 era and static landcover, multi-model ensemble, RCP 8.5 scenario, 2071-2100 era.

### Which stats?

#### Magnitude

- `ma99_hist`: Mean Annual Flow (static landcover, Maurer model, historical scenario, 1976-2005 era)
  - **note: this is for the default map view on the web app**
  - statistic units = cfs
  - difference units = NA

- `ma99_diff`: Mean Annual Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma12_diff`: Mean January Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma13_diff`: Mean February Flow
  - difference between mean of ensemble values and Maurer historical 
  - statistic units = cfs
  - difference units = percentage +/-

- `ma14_diff`: Mean March Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma15_diff`: Mean April Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma16_diff`: Mean May Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma17_diff`: Mean June Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma18_diff`: Mean July Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma19_diff`: Mean August Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma20_diff`: Mean September Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma21_diff`: Mean October Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma22_diff`: Mean November Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `ma23_diff`: Mean December Flow
  - difference between mean of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

#### Duration

- `dh1_diff`: Annual Maximum Daily Flow
  - difference between maximum of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `dl1_diff`: Annual Minimum Daily Flow
  - difference between minimum of ensemble values and Maurer historical
  - statistic units = cfs
  - difference units = percentage +/-

- `dh15_diff`: High Flow Pulse Duration
  - difference between maximum of ensemble values and Maurer historical
  - statistic units = days per year
  - difference units = absolute +/-

- `dl16_diff`: Low Flow Pulse Duration
  - difference between maximum of ensemble values and Maurer historical
  - statistic units = days per year
  - difference units = absolute +/-

#### Frequency

- `fh1_diff`: High Flood Pulse Count
  - difference between maximum of ensemble values and Maurer historical
  - statistic units = events per year
  - difference units = absolute +/-

- `fl1_diff`: Low Flood Pulse Count
  - difference between maximum of ensemble values and Maurer historical
  - statistic units = events per year
  - difference units = absolute +/-

#### Timing

- `th1_diff`: Julian Date of Annual Maximum
  - difference between mean of ensemble values and Maurer historical
  - statistic units = day of year
  - difference units = absolute +/-

- `tl1_diff`: Julian Date of Annual Minimum
  - difference between mean of ensemble values and Maurer historical
  - statistic units = day of year
  - difference units = absolute +/-



### Setup

In [206]:
import xarray as xr
import ast
import pandas as pd
import random
import numpy as np

In [207]:
vars = {"ma12":{
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        },
        "ma13":{
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        }, 
        "ma14":{
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        }, 
        "ma15":{
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        }, 
        "ma16":{
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        },
        "ma17": {
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        }, 
        "ma18": {
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        }, 
        "ma19": {
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        }, 
        "ma20": {
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        }, 
        "ma21": {
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        }, 
        "ma22": {
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        },
        "ma23": {
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
        },
        "dh1": {
                "aggregation":"maximum",
                "units":"cfs",
                "difference":"percentage"
        },
        "dl1": {
                "aggregation":"minimum",
                "units":"cfs",
                "difference":"percentage"
        },
        "dh15": {
                "aggregation":"maximum",
                "units":"days per year",
                "difference":"absolute"
        },
        "dl16": {
                "aggregation":"minimum",
                "units":"days per year",
                "difference":"absolute"
        },
        "fh1": {
                "aggregation":"maximum",
                "units":"events per year",
                "difference":"absolute"
        },
        "fl1": {
                "aggregation":"maximum",
                "units":"events per year",
                "difference":"absolute"
        },
        "th1": {
                "aggregation":"mean",
                "units":"day of year",
                "difference":"absolute"
        },
        "tl1": {
                "aggregation":"mean",
                "units":"day of year",
                "difference":"absolute"
        },
}


Load combined stats file and subset variables.

In [208]:
ds = xr.open_dataset("/import/beegfs/CMIP6/jdpaul3/hydroviz_data/maurer/nc_stats_fix/seg.nc")
ds = ds[list(vars.keys())]

Decode the dataset dimensions using the encoding attributes.

In [209]:
for dim in ds.dims:
    if ds[dim].attrs.get("encoding"):
        print(dim, " >>> ", ds[dim].attrs["encoding"])
        encoding_dict = ast.literal_eval(ds[dim].attrs["encoding"])
        decoded_labels = [encoding_dict[k] for k in ds[dim].values.astype(int)]
        ds = ds.assign_coords({dim: decoded_labels})
print(ds)

landcover  >>>  {0: 'dynamic', 1: 'static'}
model  >>>  {0: 'ACCESS1-0', 1: 'BCC-CSM1-1', 2: 'BNU-ESM', 3: 'CCSM4', 4: 'GFDL-ESM2G', 5: 'GFDL-ESM2M', 6: 'IPSL-CM5A-LR', 7: 'IPSL-CM5A-MR', 8: 'Maurer', 9: 'MIROC5', 10: 'MIROC-ESM', 11: 'MIROC-ESM-CHEM', 12: 'MRI-CGCM3', 13: 'NorESM1-M'}
scenario  >>>  {0: 'historical', 1: 'rcp26', 2: 'rcp45', 3: 'rcp60', 4: 'rcp85'}
era  >>>  {0: '1976-2005', 1: '2016-2045', 2: '2046-2075', 3: '2071-2100'}
<xarray.Dataset> Size: 3GB
Dimensions:    (landcover: 2, model: 14, scenario: 5, era: 4, stream_id: 56460)
Coordinates:
  * stream_id  (stream_id) int32 226kB 1 2 3 4 5 ... 56457 56458 56459 56460
  * landcover  (landcover) <U7 56B 'dynamic' 'static'
  * model      (model) <U14 784B 'ACCESS1-0' 'BCC-CSM1-1' ... 'NorESM1-M'
  * scenario   (scenario) <U10 200B 'historical' 'rcp26' 'rcp45' 'rcp60' 'rcp85'
  * era        (era) <U9 144B '1976-2005' '2016-2045' '2046-2075' '2071-2100'
Data variables: (12/20)
    ma12       (landcover, model, scenario, era, 

### Subset and compute ensemble

Compute a model ensemble for the static + RCP 8.5 + late century era and reference in a new dataset. Use min, mean, and max operators. Extract the Maurer historical data to a new dataset as well. Round all values to 2 decimal places.

In [210]:
ds_proj_min = ds.sel(landcover="static", scenario="rcp85", era="2071-2100").min(dim="model").assign_coords(model=["Ensemble"]).round(2)
ds_proj_mean = ds.sel(landcover="static", scenario="rcp85", era="2071-2100").mean(dim="model").assign_coords(model=["Ensemble"]).round(2)
ds_proj_max = ds.sel(landcover="static", scenario="rcp85", era="2071-2100").max(dim="model").assign_coords(model=["Ensemble"]).round(2)

In [211]:
ds_hist = ds.sel(landcover="static", model="Maurer", scenario="historical", era="1976-2005").round(2)

We need to add a variable `ma99` to the datasets: this represents our annual mean flow, which is calculated as the mean of all monthly mean flows.

In [212]:
monthly_flow_vars = [var for var in vars.keys() if var.startswith("ma")]

ds_proj_mean["ma99"] = ds_proj_mean[monthly_flow_vars].to_array(dim="var").mean("var").round(2)
ds_hist["ma99"] = ds_hist[monthly_flow_vars].to_array(dim="var").mean("var").round(2)

vars["ma99"] = {
                "aggregation":"mean",
                "units":"cfs",
                "difference":"percentage"
                }

### Sanity checks
 If we subset the projected datasets by stream ID and variable, we get a single values representing min, mean, max of the ensemble. If we subset the historical dataset by stream ID and variable, we get a single value representing the historical baseline.

In [213]:
print(ds_proj_min["ma20"].sel(stream_id=56458).values)
print(ds_proj_mean["ma20"].sel(stream_id=56458).values)
print(ds_proj_max["ma20"].sel(stream_id=56458).values)
print("\n")
print(ds_hist["ma20"].sel(stream_id=56458).values)

99.34
171.32
286.6


293.43


Check out some monthly flow values for a stream and confirm that the mean annual flow calculation worked.

In [214]:
for ds, label in zip([ds_proj_mean, ds_hist], ["Projected Mean", "Historical"]):

    print(f"{label} Monthly source values:", ds[monthly_flow_vars].sel(stream_id=56458).to_array().values)
    print(f"{label} Mean of monthly source values:", ds[monthly_flow_vars].sel(stream_id=56458).to_array().values.mean().round(2))
    print(f"{label} Dataset ma99 value:", ds["ma99"].sel(stream_id=56458).values.round(2))
    print("\n")

Projected Mean Monthly source values: [2183.38 3392.29 2656.59 1470.26  740.51  351.23  224.39  217.52  171.32
  184.4   315.27  829.56]
Projected Mean Mean of monthly source values: 1061.39
Projected Mean Dataset ma99 value: 1061.39


Historical Monthly source values: [ 680.56 1089.2  2089.3  2782.9  3957.7  3002.2  1331.2   560.28  293.43
  219.02  366.09  482.07]
Historical Mean of monthly source values: 1404.5
Historical Dataset ma99 value: 1404.5




### Calculate diffs

For each variable, use the dictionary to determine the operator and compute differences in the specified units. Store the resulting data arrays in a list for tabulation later. Print a message showing values for a test stream ID. For the `ma99` variable, also pass the historical values to the arrays list as variable `ma99_hist` for our default map view. 

In [215]:
diff_arrays = {}


for var in vars.keys():

    stream_id = 56458

    print("\nVariable: ", var)
    print(f"Test stream ID: {stream_id} ")

    agg = vars[var]["aggregation"]
    unit = vars[var]["units"]
    diff = vars[var]["difference"]

    if agg == "minimum":
        if unit == "day of year":
            diff_ds = ((ds_proj_min[var] - ds_hist[var] + (366/2)) % 366) - (366/2) # account for wrap-around at year boundary
        else:
            diff_ds = ds_proj_min[var] - ds_hist[var]

        print("     Historical: ", ds_hist[var].sel(stream_id=stream_id).values, unit)
        print("     Projected Min: ", ds_proj_min[var].sel(stream_id=stream_id).values, unit)

    if agg == "mean":
        if unit == "day of year":
            diff_ds = ((ds_proj_mean[var] - ds_hist[var] + (366/2)) % 366) - (366/2) # account for wrap-around at year boundary
        else:
            diff_ds = ds_proj_mean[var] - ds_hist[var]

        print("     Historical: ", ds_hist[var].sel(stream_id=stream_id).values, unit)
        print("     Projected Mean: ", ds_proj_mean[var].sel(stream_id=stream_id).values, unit)

    if agg == "maximum":
        if unit == "day of year":
            diff_ds = ((ds_proj_max[var] - ds_hist[var] + (366/2)) % 366) - (366/2) # account for wrap-around at year boundary
        else:
            diff_ds = ds_proj_max[var] - ds_hist[var]

        print("     Historical: ", ds_hist[var].sel(stream_id=stream_id).values, unit)
        print("     Projected Max: ", ds_proj_max[var].sel(stream_id=stream_id).values, unit)
    if diff == "percentage":
        perc_diff_ds = ((diff_ds / ds_hist[var]) * 100.0).round(2)
        diff_arrays[str(var+"_diff")] = perc_diff_ds
        # if perc_diff_ds is positive, add a + sign in message
        if perc_diff_ds.sel(stream_id=stream_id).values >= 0:
            print("     Difference (percentage): +", perc_diff_ds.sel(stream_id=stream_id).values, " %", sep="")
        else:
            print("     Difference (percentage): ", perc_diff_ds.sel(stream_id=stream_id).values, "%")
    elif diff == "absolute":
        diff_arrays[str(var+"_diff")] = diff_ds.round(2)
        # if diff_ds is positive, add a + sign in message
        if diff_ds.sel(stream_id=stream_id).values >= 0:
            print("     Difference (absolute): +", diff_ds.sel(stream_id=stream_id).values, " ",unit, sep="")
        else:
            print("     Difference (absolute): ", diff_ds.sel(stream_id=stream_id).values, unit)


diff_arrays["ma99_hist"] = ds_hist["ma99"]


Variable:  ma12
Test stream ID: 56458 
     Historical:  680.56 cfs
     Projected Mean:  2183.38 cfs
     Difference (percentage): +220.82 %

Variable:  ma13
Test stream ID: 56458 
     Historical:  1089.2 cfs
     Projected Mean:  3392.29 cfs
     Difference (percentage): +211.45 %

Variable:  ma14
Test stream ID: 56458 
     Historical:  2089.3 cfs
     Projected Mean:  2656.59 cfs
     Difference (percentage): +27.15 %

Variable:  ma15
Test stream ID: 56458 
     Historical:  2782.9 cfs
     Projected Mean:  1470.26 cfs
     Difference (percentage):  -47.17 %

Variable:  ma16
Test stream ID: 56458 
     Historical:  3957.7 cfs
     Projected Mean:  740.51 cfs
     Difference (percentage):  -81.29 %

Variable:  ma17
Test stream ID: 56458 
     Historical:  3002.2 cfs
     Projected Mean:  351.23 cfs
     Difference (percentage):  -88.3 %

Variable:  ma18
Test stream ID: 56458 
     Historical:  1331.2 cfs
     Projected Mean:  224.39 cfs
     Difference (percentage):  -83.14 %

Var

### Convert Arrays to Dataframe

Create a dataframe with stream IDs as index, and use the arrays to populate the rest of the columns. 

In [216]:
df = pd.DataFrame(diff_arrays, index=diff_arrays["ma20_diff"].stream_id.values)
df

,ma12_diff,ma13_diff,ma14_diff,ma15_diff,ma16_diff,ma17_diff,ma18_diff,ma19_diff,ma20_diff,ma21_diff,...,dh1_diff,dl1_diff,dh15_diff,dl16_diff,fh1_diff,fl1_diff,th1_diff,tl1_diff,ma99_diff,ma99_hist
1,273.089996,513.760010,544.340027,-63.549999,-76.330002,-23.990000,-20.820000,-12.410000,-1.860000,-37.439999,...,-20.209999,167.119995,-0.99,-7.16,7.44,18.50,-42.130001,-175.029999,-13.540000,117.349998
2,202.149994,277.250000,84.849998,-48.470001,-38.369999,-25.730000,-28.350000,-30.180000,-22.940001,-33.169998,...,-12.550000,-43.369999,-0.04,-2.46,4.04,4.33,-43.590000,-13.800000,12.460000,374.309998
3,200.699997,298.730011,153.520004,-66.989998,-70.160004,-35.759998,-30.260000,-26.430000,-15.280000,-42.410000,...,-11.970000,26.799999,1.92,-7.11,1.17,5.00,-13.990000,-145.470001,-6.220000,870.780029
4,196.660004,284.829987,144.380005,-65.660004,-69.110001,-36.009998,-30.549999,-27.020000,-15.410000,-42.070000,...,-12.190000,5.340000,1.35,-4.63,1.13,4.04,-37.500000,-109.510002,-5.440000,918.460022
5,201.410004,274.829987,83.199997,-48.709999,-39.279999,-26.900000,-29.370001,-30.760000,-23.469999,-34.740002,...,-13.680000,-43.459999,0.75,-1.71,3.70,2.90,-45.369999,-8.300000,11.590000,447.839996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56457,-28.180000,-35.509998,-63.540001,-66.519997,-67.529999,-67.500000,-64.589996,-51.299999,-38.660000,-41.000000,...,186.199997,-82.440002,27.67,-21.32,0.03,1.34,62.250000,-15.120000,-52.689999,21.010000
56458,220.820007,211.449997,27.150000,-47.169998,-81.290001,-88.300003,-83.139999,-61.180000,-41.610001,-15.810000,...,174.320007,-57.369999,9.67,-9.00,1.04,1.04,-81.029999,-7.990000,-24.430000,1404.500000
56459,36.299999,22.420000,2.340000,-16.610001,-28.430000,-38.750000,-31.799999,-33.160000,-37.639999,-22.670000,...,74.459999,-41.720001,1.80,-2.30,0.30,1.27,73.220001,1.370000,5.520000,3300.350098


### QC

Pick 10 random stream IDs from the original dataset, reprocess, and test results against the dataframe.

In [217]:
# Pick 10 random stream IDs from the original dataset, and test results against the dataframe.
# We will just use the June monthly mean flow to test.

for _ in range(10):
    stream_id = random.choice(ds.stream_id.values.tolist())
    print(f"Testing stream ID: {stream_id} ")

    var = "ma20"
    print("     Historical value:", ds_hist[var].sel(stream_id=stream_id).values)
    print("     Projected Mean value:", ds_proj_mean[var].sel(stream_id=stream_id).values)
    print("     Calculated difference (percentage):", df.loc[stream_id, var+"_diff"])
    calc_diff = ((ds_proj_mean[var].sel(stream_id=stream_id) - ds_hist[var].sel(stream_id=stream_id)) / ds_hist[var].sel(stream_id=stream_id)) * 100.0
    print("     Verified difference (percentage):", calc_diff.values)
    

Testing stream ID: 36600 
     Historical value: 1.5
     Projected Mean value: 0.78
     Calculated difference (percentage): -48.0
     Verified difference (percentage): -48.00000190734863
Testing stream ID: 40630 
     Historical value: 0.69
     Projected Mean value: 1.8
     Calculated difference (percentage): 160.87
     Verified difference (percentage): 160.86955070495605
Testing stream ID: 54313 
     Historical value: 57.6
     Projected Mean value: 34.12
     Calculated difference (percentage): -40.76
     Verified difference (percentage): -40.76388776302338
Testing stream ID: 1601 
     Historical value: 258.57
     Projected Mean value: 94.32
     Calculated difference (percentage): -63.52
     Verified difference (percentage): -63.522446155548096
Testing stream ID: 17120 
     Historical value: 525.57
     Projected Mean value: 216.64
     Calculated difference (percentage): -58.78
     Verified difference (percentage): -58.779990673065186
Testing stream ID: 4063 
     Hist

### Save to CSV

Rename the index to "stream_id" so we can crosswalk this data into the shapefile, then save.

In [218]:
# Rename the index to "stream_id"
df.index.name = "stream_id"
df.to_csv("stats_diff.csv", index=True)